### Bipartite Network

In [21]:
import numpy as np
import pandas as pd
import dask.dataframe as dd
import matplotlib.pyplot as plt

import torch
from torch_geometric.data import Data
from torch_geometric.utils import from_scipy_sparse_matrix
from scipy.sparse import coo_matrix
from torch_geometric.nn import GCNConv
import torch.nn.functional as F
from torch_geometric.data import DataLoader

import ast

from scipy.sparse import load_npz

In [8]:
urls_df = pd.read_csv("C:/Users/Raphaela/Documents/MA_Studium/4_Semester/MA_Thesis/02_data/url_titles_domains.csv")

In [9]:
urls_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5195 entries, 0 to 5194
Data columns (total 13 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   id               5195 non-null   int64  
 1   url              5195 non-null   object 
 2   start_date       5195 non-null   object 
 3   end_date         5195 non-null   object 
 4   title            5195 non-null   object 
 5   domain           5195 non-null   object 
 6   pc1              5195 non-null   float64
 7   chat_idx         5195 non-null   int64  
 8   start_year       5195 non-null   int64  
 9   start_month      5195 non-null   int64  
 10  start_day        5195 non-null   int64  
 11  start_hour       5195 non-null   int64  
 12  title_embedding  5195 non-null   object 
dtypes: float64(1), int64(6), object(6)
memory usage: 527.7+ KB


In [23]:
urls_df

,id,url,start_date,end_date,title,domain,pc1,chat_idx,start_year,start_month,start_day,start_hour,title_embedding,domain_id
0,16813965,https://rumble.com/v11uxz5-president-trump-bus...,2022-04-22 07:29:05.000000,2022-04-22 07:29:05.000000,v11uxz5 president trump busted piers morgan. m...,rumble.com,0.162506,65541,2022,4,22,7,[-5.52624790e-03 4.81570959e-02 3.91092412e-...,0
1,8937464,https://gender.news/2019-05-22-democrats-passa...,2019-06-18 05:40:34.000000,2019-06-18 05:40:34.000000,2019 05 22 democrats passage of equality act a...,gender.news,0.211351,196615,2019,6,18,5,[-2.83298679e-02 8.94291997e-02 -1.82891339e-...,1
2,12896181,https://amgreatness.com/2022/06/28/top-gun-mav...,2022-06-29 18:03:58.000000,2023-08-21 20:27:56.389222,top gun maverick a boomer fantasy,amgreatness.com,0.388267,196616,2022,6,29,18,[-5.35068586e-02 4.42830920e-02 -3.57456878e-...,2
3,19933646,https://nypost.com/2023/01/06/passenger-hauled...,2023-01-06 19:34:07.000000,2023-01-06 19:34:07.000000,passenger hauled of air canada flight after al...,nypost.com,0.638970,65544,2023,1,6,19,[-1.34245725e-03 7.86104947e-02 2.65778205e-...,3
4,10915134,https://nypost.com/2023/06/21/former-nfl-steel...,2023-06-21 07:35:00.000000,2023-08-29 17:08:37.212656,former nfl steelers linebacker clark haggans d...,nypost.com,0.638970,196617,2023,6,21,7,[-5.68774752e-02 5.96293807e-02 -1.87179640e-...,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5190,3849445,https://gript.ie/on-this-day-22-january-1925-b...,2022-01-22 12:12:39.000000,2023-06-02 04:42:19.071467,on this day 22 january 1925 birth of raymond c...,gript.ie,0.326155,277953,2022,1,22,12,[ 2.88018305e-02 4.00173515e-02 -5.48276305e-...,24
5191,5278095,https://thefederalist.com/2022/09/15/l-a-schoo...,2022-09-15 13:59:18.000000,2022-09-15 13:59:18.000000,?utm source=rss&utm medium=rss&utm campaign=l ...,thefederalist.com,0.257333,146887,2022,9,15,13,[-5.81897609e-03 3.22640277e-02 2.24588346e-...,10
5192,17078374,http://feeds.foxnews.com/~r/foxnews/world/~3/b...,2021-05-25 16:37:58.000000,2021-05-25 16:37:58.000000,congo volcano eruption death toll,feeds.foxnews.com,0.587644,15820,2021,5,25,16,[-5.21009509e-03 -1.22024317e-03 2.94684935e-...,45
5193,1911447,https://nationalfile.com/pennsylvania-gop-esta...,2022-09-25 21:18:59.000000,2023-07-25 12:30:15.280887,pennsylvania gop establishment fundraises for ...,nationalfile.com,0.118408,277965,2022,9,25,21,[ 2.72592790e-02 -1.96310245e-02 3.39954682e-...,131


In [24]:
print(urls_df['title_embedding'].head())

0    [-5.52624790e-03  4.81570959e-02  3.91092412e-...
1    [-2.83298679e-02  8.94291997e-02 -1.82891339e-...
2    [-5.35068586e-02  4.42830920e-02 -3.57456878e-...
3    [-1.34245725e-03  7.86104947e-02  2.65778205e-...
4    [-5.68774752e-02  5.96293807e-02 -1.87179640e-...
Name: title_embedding, dtype: object


In [33]:
# Determine the maximum length of embeddings
max_length = max(len(embedding) for embedding in urls_df['title_embedding'])

# Pad each embedding to the max_length
padded_embeddings = [
    F.pad(torch.tensor(embedding, dtype=torch.float32), (0, max_length - len(embedding)))
    for embedding in urls_df['title_embedding']
]

# Stack padded embeddings into a tensor
url_features = torch.stack(padded_embeddings)

TypeError: new(): invalid data type 'str'

In [34]:
# Stack embeddings as a tensor
url_features = torch.stack([torch.tensor(embedding, dtype=torch.float32) for embedding in urls_df['title_embedding']])

TypeError: new(): invalid data type 'str'

In [11]:
df = pd.read_parquet('../02_data/data_archive/raphaela/chats.parquet')

In [7]:
df

,db_index,name,type,description,id
19,12250,drsambaileyofficial,ChatType.CHANNEL,None,0
24,12255,worlddoctorsalliance,ChatType.CHANNEL,None,1
45,12276,real_hero_english,ChatType.CHANNEL,None,2
57,12288,corona_investigative,ChatType.CHANNEL,None,3
59,12290,en_2020news,ChatType.CHANNEL,None,4
...,...,...,...,...,...
570698,865501,thelastdayonearthwillbejstasgray,ChatType.CHANNEL,None,65333
571722,871874,days_of_the_speed_on_baykal_ice,ChatType.CHANNEL,None,65334
571760,872159,whateverhappendtotheyounggirlhrt,ChatType.CHANNEL,None,65335
572232,874570,OpinionTvAnimationsAllTheWay,ChatType.CHANNEL,None,65336


In [17]:
chat_url_share = load_npz('../02_data/data_archive/raphaela/chat_url_shares.npz') 

In [32]:
chat_url_share[:10]

TypeError: 'coo_matrix' object is not subscriptable

In [18]:
# Step 2: Merge URL and Domain Information with Chats
# Get unique domain indices
domain_ids = urls_df['domain'].factorize()[0]
urls_df['domain_id'] = domain_ids

# Create edges from sparse matrix
chat_indices, url_indices = chat_url_share.nonzero()  # Chats and URLs with edges

# Step 3: Define Node Features
# URL features
url_features = torch.stack([torch.tensor(embedding) for embedding in urls_df['title_embedding']])

# Chat features (dummy features if not present)
chat_features = torch.randn(len(df), url_features.size(1))  # Random embeddings

# Step 4: Define Edge Index and Graph Data
edge_index = torch.tensor([chat_indices, url_indices], dtype=torch.long)

# Stack features
node_features = torch.cat([chat_features, url_features], dim=0)

# Define target as pc1, grouped by domain
domain_pc1 = torch.tensor(urls_df.groupby('domain_id')['pc1'].first().values)

# Create PyTorch Geometric Data object
graph_data = Data(x=node_features, edge_index=edge_index, y=domain_pc1)

TypeError: new(): invalid data type 'str'

In [ ]:
# Define GNN Model for Prediction
class GCN(torch.nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim):
        super(GCN, self).__init__()
        self.conv1 = GCNConv(input_dim, hidden_dim)
        self.conv2 = GCNConv(hidden_dim, output_dim)

    def forward(self, data):
        x, edge_index = data.x, data.edge_index
        x = F.relu(self.conv1(x, edge_index))
        x = self.conv2(x, edge_index)
        return x

In [ ]:
# Initialize model
model = GCN(input_dim=node_features.size(1), hidden_dim=64, output_dim=1)
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
criterion = torch.nn.MSELoss()

In [ ]:
# Step 6: Train Model
model.train()
for epoch in range(200):
    optimizer.zero_grad()
    out = model(graph_data)
    loss = criterion(out[graph_data.y.nonzero(as_tuple=True)[0]], graph_data.y)
    loss.backward()
    optimizer.step()
    print(f"Epoch {epoch+1}, Loss: {loss.item()}")